__BIOBSS - Polar reader__

_This notebook includes guidelines to save Polar data files as csv file and and synchronize the signal from different sensors._

In [ ]:
#Import BIOBSS and the other required packages

#import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import biobss from local to run without installing
import sys
sys.path.append("../")
import biobss

# Table of Contents
1.[Polar Sample Data](#polar)<br>
2.[Timestamp Conversion](#polar_time)<br>
3.[Synchronization of Signals from Different Sensors](#polar_sync)<br>
4.[Data Segmentation Based on Events](#polar_event)<br>
5.[Loading and Saving Alternatives](#polar_files)<br>

### __Polar Sample Data__
<a id="polar"></a>

Polar sample data is provided as txt files in BIOBSS\sample data\Polar_files\txt_files. A seperate txt file is available for each sensor. The sampling rates are 52 Hz, 52 Hz, 50 Hz and 55 Hz for ACC, GYRO, MAGN and PPG sensors respectively. 

In [ ]:
#Define the paths for the sample files
polar_dir=os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir),"sample_data","Polar_files")
txt_dir=os.path.join(polar_dir,'txt_files')
csv_dir=os.path.join(polar_dir,'csv_files')
pkl_dir=os.path.join(polar_dir,'pkl_files')

Txt files can be saved as csv files into BIOBSS\sample_data\Polar_files\csv_files. If required, the files can be renamed using the function ___rename_csv___.

In [ ]:
#Save txt files as csv files
biobss.reader.polar_format.txt_to_csv(txt_dir)

#Rename csv files using the record_id
biobss.reader.polar_format.rename_csv(csv_dir)

### __Timestamp Conversion__
<a id="polar_time"></a>

Timestamps in iso format can be converted to milliseconds for a specific file. If required, the reference timestamp can be provided by the parameter __start_time__. Otherwise, the first timestamp is used as reference.

In [ ]:
#Convert timestamps from iso format to milliseconds for a specific file
record_id='sample_record'
filename='sample_record_PPG.csv'
filepath=os.path.join(csv_dir,record_id,filename)
df= pd.read_csv(filepath)
time_msec=biobss.reader.polar_format.timestamp_to_msec(df['Phone timestamp'])

The ___add_record_time___ function can be used to update the selected csv files by adding a 'Time_record (ms)' column corresponding to time points in ms. The time points are calculated referenced to the earliest timestamp for all sensors.

In [ ]:
#Convert timestamps from iso format to milliseconds for all files in a record
record_id='sample_record'
csv_subdir=os.path.join(csv_dir,record_id) #This time, the path should be defined for a specific record.
biobss.reader.add_record_time(csv_dir=csv_subdir,marker=True)

### __Synchronization of Signals from Different Sensors__
<a id="polar_sync"></a>

Signals from different sensors can be synchronized using the time points. First, a time array is generated by detecting the overlapping time points for all sensors. Then, the 'Time_record (ms)' column of each sensor is interpolated on the generated time array. If required, signals can also be resampled at this stage. 

In [ ]:
#Generate a time array considering the overlapping time points for all sensors. 
#The path should be defined for a specific record.
#If required, save the generated time array as a txt file by setting save_file=True
time_list=biobss.reader.calculate_sync_time(csv_dir=csv_subdir,time_step=1,marker=True)

In [ ]:
#Synchronize the signals from different sensors by interpolating the signals for the given time_list.
#Resample the signals if required by providing resampling_rate 
#Set save_files=True to save the synchronized signals as a csv file
data=biobss.reader.synchronize_signals(csv_subdir,time_list=time_list,resampling_rate=100,save_files=True)

### __Data Segmentation Based on Events__
<a id="polar_event"></a>

If required, sensor data can be segmented for different events. For this purpose, a marker file is needed. Using Polar Sensor Logger App, markers can be placed during recording and a marker file can be exported. BIOBSS provides ___marker_to_events___ function to generate an event list from the marker timestamps. 

In [ ]:
#Segment the synchronized signals for events
filename = 'sync_ACC_PPG_MAGN_GYRO.csv'
filepath=os.path.join(csv_dir, record_id, filename)
markerpath=os.path.join(csv_dir, record_id, 'MARKER_'+record_id+'.csv')

data=biobss.reader.marker_to_events(filepath,markerpath,['rest'])

In [ ]:
#The event list can also be saved as txt file for later use
out_path=os.path.join(csv_dir, record_id, 'event_list.txt' )
biobss.reader.marker_to_events(filepath,markerpath,['rest'],out_path,save_file=True)

### __Loading and Saving Alternatives__
<a id="polar_files"></a>

BIOBSS provides loading and saving alternatives for the Polar sensor files. Csv files can be read into dictionaries using the ___polar_csv_reader___ function by setting the signal_type parameter. 

Csv files can also be saved as pkl files using the ___csv_to_pkl___ function.

In [ ]:
#Read csv files into a dictionary
filename='sample_record_PPG.csv'
filepath=os.path.join(csv_dir,record_id,filename)
data=biobss.reader.polar_csv_reader(filepath,signal_type='PPG')
print(data.keys())

#Save signals as pkl files
biobss.reader.csv_to_pkl(csv_dir)